# First Things First

Welcome to our very first introduction to *Tensor Fox*, a specialized library made to deal with 3D tensors, with focus on the CPD (Canonical Polyadic Decomposition). In order to have everything working properly, all files of Tensor Fox must be in the same folder of your program. Another option is to add all modules to your Python path, if more convenient. To be able to use Tensor Fox you will need the following modules:

- numpy

- scipy

- time

- matplotlib

- numba

- decimal

Also make sure Numpy is using a nice version of BLAS. That is all! Tensor Fox is read to go! Let's start importing Tensor Fox and other necessary modules.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import TensorFox as tfx

# Creating Tensors and Getting Information 

Let's create a little tensor $T$ just to see how Tensor Fox works at its basics. It is important to notice that TensorFox only works with tensors as 3D numpy arrays. 

Our convention is that $T[i,j,k]$ refers to the $i$-th row, $j$-column and $k$-section (frontal slice) of $T$. For instance, consider the tensor defined above (the frontal slices of $T$ are showed)

$$T = \left\{ \left[
\begin{array}{cc}
    0 & 1\\
    2 & 3
\end{array}
\right], \quad
\left[
\begin{array}{cc}
    4 & 5\\
    6 & 7
\end{array}
\right] \right\}.$$

Since Numpy's convention is different from ours, we have a special command to print $T$ by slices. This is the function **showtens**. Using the **print** function will show a different $T$ and this can lead to confusion.

In [2]:
# Create and print the tensor, which is 2 x 2 x 2.
m = 2
T = np.zeros((m, m, m))
s = 0

for k in range(m):
    for i in range(m):
        for j in range(m):
            T[i,j,k] = s
            s += 1

print('Our view of T:')
print('--------------')
Tsize = np.linalg.norm(T)            
tfx.disp.showtens(T)
print()
print('Numpy view of T:')
print('----------------')
print(T)

Our view of T:
--------------
[[0. 1.]
 [2. 3.]]

[[4. 5.]
 [6. 7.]]


Numpy view of T:
----------------
[[[0. 4.]
  [1. 5.]]

 [[2. 6.]
  [3. 7.]]]


There are lots of questions we can make about a given tensor $T$: what are its maximum and minimum entries, what are the dimensions of $T$, the rank and multirank, etc.

Even in the case of the simple tensor above we can't know all these answers in advance. The function **infotens** tries to compute all these informations for your convenience. This function is useful for small tensors, but remember that finding the rank is a NP-hard task, so don't abuse. 

In [3]:
# Let's see some information about T.
tfx.disp.infotens(T)

T is a tensor of dimensions 2 x 2 x 2

|T| = 11.832159566199232

max(T) = 7.0

min(T) = 0.0

E[T] = 3.5

E[|T|] = 3.5

1 <= rank(T) <= 4

Computing multilinear rank...
------------------------------------
Estimated multirank(T) = 2 , 2 , 2
|T - (U1, U2, U3)*S|/|T| = 0.0

Computing rank...
Start searching for rank
Stops at r = 4  or less
-----------------------------
Testing r = 4
Estimated rank(T) = 3
|T - T_approx|/|T| = 2.0717920134598587e-07



# Computing the CPD

Now let's turn to the most important tool of Tensor Fox, the computation of the CPD. As the previous function hinted, $T$ should have rank 3 (but don't believe blindly in this function). We can compute the corresponding CPD with the function **cpd**.

In [4]:
# Compute the CPD of T, assuming T has rank 2.
r = 3
Lambda, X, Y, Z, T_approx, output = tfx.cpd(T, r)

# Show the CPD computed. 
print('Lambda =\n',Lambda)
print()
print('X =\n', X)
print()
print('Y =\n', Y)
print()
print('Z =\n', Z)
print()

# Show the coordinate representation of this CPD.
print('T_approx =')
tfx.disp.showtens(T_approx)

# Show relative error of this approximation.
print('|T - T_approx|/|T| =', output.rel_error)

Lambda =
 [3.18468731 8.07478364 4.87031198]

X =
 [[0.59176792 0.0478316  0.9999932 ]
 [0.80610839 0.99885541 0.0036865 ]]

Y =
 [[0.02366698 0.77527723 0.755659  ]
 [0.9997199  0.6316211  0.65496525]]

Z =
 [[ 0.54454496  0.31462404 -0.0321988 ]
 [ 0.83873165  0.94921637  0.99948148]]

T_approx =
[[-2.62842795e-06  1.00000270e+00]
 [ 2.00000231e+00  2.99999738e+00]]

[[4.00000021 4.99999899]
 [5.99999892 7.00000055]]

|T - T_approx|/|T| = 4.545902930936984e-07


If it is not convenient to use the array $\Lambda$, we can convert everything to the format $X, Y, Z$ as showed below. The factor aren't unit norm as before but they are constructed in order to have the same norm, that is, $\| X[:,l] \| = \| Y[:,l] \| = \| Z[:,l] \|$ for $l = 0,1,2$.

In [5]:
X, Y, Z = tfx.aux.denormalize(Lambda, X, Y, Z)

print('X =\n', X)
print()
print('Y =\n', Y)
print()
print('Z =\n', Z)
print()

X =
 [[0.87064347 0.09596035 1.69505051]
 [1.18599367 2.00391637 0.00624884]]

Y =
 [[0.03482024 1.55537099 1.28088887]
 [1.47084622 1.26716625 1.11020673]]

Z =
 [[ 0.80116631  0.63120273 -0.05457897]
 [ 1.23399092  1.90432989  1.69418311]]

